# Donwload data

In [ ]:
!wget https://food-x.s3.amazonaws.com/annot.tar
!wget https://food-x.s3.amazonaws.com/train.tar
!wget https://food-x.s3.amazonaws.com/val.tar

!tar -xvf annot.tar -C /Data
!tar -xvf train.tar -C /Data
!tar -xvf val.tar -C /Data

# Preprocessing

In [ ]:
import os
import glob
import cv2
import numpy as np
from PIL import Image
import torch
from tqdm import tqdm
import pandas as pd

train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print("running on:", device)


# class list pre processing
def add_string(file, string):
    with open(file, 'r') as original: data = original.read()
    if data.startswith(string): return
    with open(file, 'w') as modified: modified.write(string + data)


add_string('Data/annot/train_info.csv', 'class,image\n')
add_string('Data/annot/val_info.csv', 'class,image\n')

# Directory:
trainDirectory = 'Data/train_set'
testDirectory = 'Data/val_set'

processedTrainDirectory = 'Data/processedData/processed_train_set'
processedTestDirectory = 'Data/processedData/processed_test_set'
processedValDirectory = 'Data/processedData/processed_val_set'

# Creation of the directory if they dont exist
os.makedirs(processedTrainDirectory, exist_ok=True)
os.makedirs(processedTestDirectory, exist_ok=True)
os.makedirs(processedValDirectory, exist_ok=True)

# load images classes from "train_info.csv"
trainInfo = pd.read_csv('Data/annot/train_info.csv')
testInfo = pd.read_csv('Data/annot/val_info.csv')

trainClasses = trainInfo['class'].unique()
testClasses = testInfo['class'].unique()

# create folders for each train class
for trainClass in trainClasses:
    os.makedirs(os.path.join(processedTrainDirectory, str(trainClass)), exist_ok=True)

# create folders for each test class
for testClass in testClasses:
    os.makedirs(os.path.join(processedTestDirectory, str(testClass)), exist_ok=True)

# create folders for each validation class (using the train classes)
for valClass in trainClasses:
    os.makedirs(os.path.join(processedValDirectory, str(valClass)), exist_ok=True)


def process_images(input_dir, output_dir, set_type):
    image_path_pattern = os.path.join(input_dir, "*.jpg")
    image_paths = glob.glob(image_path_pattern)

    pbar = tqdm(total=len(image_paths), desc='Processing', unit='frame')

    for image_path in image_paths:
        image = cv2.imread(image_path)

        # get image class
        if set_type == "train":
            imageClass = trainInfo[trainInfo['image'] == os.path.basename(image_path)]['class'].values[0]
        else:
            imageClass = testInfo[testInfo['image'] == os.path.basename(image_path)]['class'].values[0]

        # Check if the image is in RBG format
        if (len(image.shape) != 3):
            print("is not RGB")

        # Resize of the images to a common size
        image = cv2.resize(image, (256, 256))

        # path to save the images
        base_filename = os.path.basename(image_path)
        processed_image_path = os.path.join(output_dir, str(imageClass), base_filename)

        # saving the images
        cv2.imwrite(processed_image_path, image)

        pbar.update(1)


def valSet():
    # take 3% of the train set and create a vaidation set
    image_path_pattern = os.path.join(processedTrainDirectory, "*.jpg")
    image_paths = glob.glob(image_path_pattern)

    # 3% of the train set drawn randomly
    val_set = np.random.choice(image_paths, int(len(image_paths) * 0.03), replace=False)

    # path to save the images
    for image_path in val_set:
        # get image class from train set
        imageClass = trainInfo[trainInfo['image'] == os.path.basename(image_path)]['class'].values[0]

        base_filename = os.path.basename(image_path)
        processed_image_path = os.path.join(processedValDirectory, str(imageClass), base_filename)

        # move the images to the test set
        os.rename(image_path, processed_image_path)


print("Processing train set")
process_images(trainDirectory, processedTrainDirectory, "train")

print("Processing test set")
process_images(testDirectory, processedTestDirectory, "test")

# Creation of the validation set
print("Creating validation set")
valSet()


# Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        # Define the feature extraction part of the network
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Define the classification part of the network
        self.fc1 = nn.Linear(32 * 4 * 4, 256)  # Adjusted size due to pooling
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

        self.dropout = nn.Dropout(0.5)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        # Feature extraction
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = F.relu(self.conv4(x))
        x = self.pool4(x)

        # Flatten the tensor for the fully connected layers
        x = x.view(x.size(0), -1)

        # Classification
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

    def trainStep(self, x, y):
        self.optimizer.zero_grad()
        out = self.forward(x)
        loss = self.criterion(out, y)
        loss.backward()
        self.optimizer.step()
        return loss

# Training

In [ ]:
import torch
import torchvision.datasets
from net import Net
from torchsummary import summary
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

def main():
    train_on_gpu = torch.cuda.is_available()
    device = torch.device("cuda:0" if train_on_gpu else "cpu")
    print("running on: ", device)

    net = Net(num_classes=251)
    summary(net, (3, 64, 64))
    net.to(device)

    lossOvertime = []
    accuracyOvertime = []

    # Define data transformations pipeline
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((64, 64))
    ])

    trainSet = torchvision.datasets.ImageFolder(root='./Data/processedData/processed_train_set', transform=transforms)
    testSet = torchvision.datasets.ImageFolder(root='./Data/processedData/processed_test_set', transform=transforms)

    trainLoader = DataLoader(trainSet, batch_size=64, shuffle=True, num_workers=2)
    testLoader = DataLoader(testSet, batch_size=64, shuffle=True, num_workers=2)

    epochs = 5

    for epoch in range(epochs):
        running_loss = 0.0

        for i, data in tqdm(enumerate(trainLoader, 0), total=len(trainLoader)):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            loss = net.trainStep(inputs, labels)
            running_loss += loss.item()

        lossOvertime.append(running_loss)


        # check accuracy on test set
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testLoader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        accuracyOvertime.append(accuracy)
        print(f"Epoch {epoch + 1}, loss: {running_loss}, accuracy: {accuracy}")

        # save model
        torch.save(net.state_dict(), f"model_{epoch}.pth")


if __name__ == '__main__':
    main()